## SQL Analysis: Athlete Performance Across Age, Gender, and Elevation

The notebook contains all SQL analysis made using th 'track_jumps.db' database. 

In [29]:
import os
import sqlite3
import pandas as pd

In [30]:
conn = sqlite3.connect("../track_jumps.db")

In [31]:
pd.read_sql(
    "SELECT name FROM sqlite_master WHERE type='table';",
    conn
)

,name
0,sqlite_sequence
1,dim_athletes
2,dim_events
3,dim_meets
4,fact_performances


In [32]:
pd.read_sql("""
SELECT *
FROM dim_meets
LIMIT 100;
""", conn)

,meet_id,venue,country,latitude,longitude,elevation,is_indoor
0,5411,Salamanca (ESP),None,-33.371448,-70.691401,506.0,0
1,5412,San Juan (PUR),None,18.465299,-66.116666,26.0,0
2,5413,"Budapest Sportcsarnok, Budapest (HUN) (i)",None,47.450687,19.091052,107.0,1
3,5414,"Boudewijnstadion, Bruxelles (BEL)",None,50.866268,4.349083,18.0,0
4,5415,Stockholm (SWE),None,59.358706,17.873668,21.0,0
...,...,...,...,...,...,...,...
95,5506,"Deveti Septemvri Stadium, Plovdiv (BUL)",None,42.141854,24.749930,169.0,0
96,5507,Zürich (SUI),None,47.363835,8.545057,400.0,0
97,5508,Kiev (URS),None,55.754671,37.615825,142.0,0
98,5509,Sulingen (GER) (i),None,52.682837,8.801892,49.0,1


The country column exist in the schema to support future analysis, however, country-level venue data was left null at this time. Latitiude and longitude are included for future weather based analysis.

In [12]:
venues_df = pd.read_csv("../data/interim/venues/geocoded_venues.csv")
venues_df.columns

Index(['Venue', 'latitude', 'longitude', 'elevation', 'geocode_result'], dtype='object')

### Query 1: Performance Counts by Gender and Event

Establishes how much data is available for future queries to be run against.

In [56]:
query1 = """
SELECT 
    e.event_name, 
    e.gender, 
    count(*) AS performance_count 
FROM fact_performances f 
JOIN dim_events e 
    ON f.event_id = e.event_id 
GROUP BY e.event_name, e.gender 
ORDER BY performance_count DESC;
"""
    
pd.read_sql(query1, conn)  
    

,event_name,gender,performance_count
0,high jump,M,9861
1,long jump,M,9821
2,triple jump,M,8633
3,triple jump,F,7419
4,long jump,F,3981
5,high jump,F,3829


## Peak Performance Age by Event and Gender
#### At what age do athletes typically achieve their best performances? How does this differ by event and gender?
##### How I answered this question:
- Join fact_performance and dim_events
- Group by event and gender
- Calculate:
 - average age
 - youngest age
 - oldest age
##### _Athough this is not a true peak performance age, it does demonstrate where top performances appear in the data._

In [57]:
query2 = """
SELECT 
    e.event_name,
    e.gender,
    Round(AVG(f.age), 1) AS avg_age,
    MIN(f.age) AS youngest_age,
    COUNT(*) AS performances
FROM fact_performances f
JOIN dim_events e
    ON f.event_id = e.event_id
GROUP BY e.event_name, e.gender
ORDER BY avg_age;"""

pd.read_sql(query2, conn)


,event_name,gender,avg_age,youngest_age,performances
0,long jump,M,24.3,11,9821
1,high jump,M,24.4,16,9861
2,high jump,F,25.1,15,3829
3,triple jump,M,25.1,15,8633
4,long jump,F,25.4,14,3981
5,triple jump,F,25.8,12,7419


# Elevation and Performance
#### Query 3 explores the effect altitude may have on performance. Elevation defined by the following criteria: Sea level 0m, Low Elevation <1000m, and High Elevation is ≥1000. 

In [67]:
query3 = """
SELECT
    e.event_name,
    e.gender,
    CASE
        When m.elevation >= 1000 THEN 'High Elevation (≥1000m)'
        When m.elevation = 0 THEN 'Sea-level'
        ELSE 'Low Elevation (<1000m)'
    END AS elevation_group,
    ROUND(AVG(f.mark), 2) AS avg_mark,
    COUNT(*) AS performances
FROM fact_performances f
JOIN dim_events e
    ON f.event_id = e.event_id
JOIN dim_meets m
    ON f.meet_id = m.meet_id
GROUP BY e.event_name, e.gender, elevation_group;
"""

pd.read_sql(query3, conn)

,event_name,gender,elevation_group,avg_mark,performances
0,high jump,F,High Elevation (≥1000m),1.97,146
1,high jump,F,Low Elevation (<1000m),1.97,3615
2,high jump,F,Sea-level,1.97,68
3,high jump,M,High Elevation (≥1000m),2.29,328
4,high jump,M,Low Elevation (<1000m),2.29,9289
5,high jump,M,Sea-level,2.29,244
6,long jump,F,High Elevation (≥1000m),6.92,163
7,long jump,F,Low Elevation (<1000m),6.89,3731
8,long jump,F,Sea-level,6.93,87
9,long jump,M,High Elevation (≥1000m),8.17,684


Query 4 takes the previous query and adds the MAX and MIN performance averages at each elevation.

In [68]:
query4 = """
SELECT
    e.event_name,
    e.gender,
    CASE
        When m.elevation >= 1000 THEN 'High Elevation (≥1000m)'
        When m.elevation = 0 THEN 'Sea-level'
        ELSE 'Low Elevation (<1000m)'
    END AS elevation_group,
    ROUND(AVG(f.mark), 2) AS avg_mark,
    ROUND(MAX(f.mark), 2) AS best_mark,
    ROUND(MIN(f.mark), 2) AS low_mark,    
    COUNT(*) AS performances
FROM fact_performances f
JOIN dim_events e
    ON f.event_id = e.event_id
JOIN dim_meets m
    ON f.meet_id = m.meet_id
GROUP BY e.event_name, e.gender, elevation_group
HAVING COUNT (*) >=100
ORDER BY e.event_name, e.gender, elevation_group;
"""

pd.read_sql(query4, conn)

,event_name,gender,elevation_group,avg_mark,best_mark,low_mark,performances
0,high jump,F,High Elevation (≥1000m),1.97,2.06,1.92,146
1,high jump,F,Low Elevation (<1000m),1.97,2.09,1.90,3615
2,high jump,M,High Elevation (≥1000m),2.29,2.40,2.25,328
3,high jump,M,Low Elevation (<1000m),2.29,2.45,2.25,9289
4,high jump,M,Sea-level,2.29,2.38,2.25,244
5,long jump,F,High Elevation (≥1000m),6.92,7.63,6.59,163
6,long jump,F,Low Elevation (<1000m),6.89,7.49,6.55,3731
7,long jump,M,High Elevation (≥1000m),8.17,8.99,7.75,684
8,long jump,M,Low Elevation (<1000m),8.15,8.95,7.75,8900
9,long jump,M,Sea-level,8.14,8.60,7.76,237


Query 5 
Phase 1
Top 10 athletes that have competed at all levels (U18, U20, Senior). These queries are not used in this project but are parked here for future analysis.

In [ ]:
query_phase1 = """
WITH ranked_athletes AS (
    SELECT
        a.athlete_id,
        a.name AS athlete,
        e.event_name,
        e.gender,
        MAX(f.mark) AS best_mark
    FROM fact_performances f
    JOIN dim_athletes a
        ON f.athlete_id = a.athlete_id
    JOIN dim_events e
        ON f.event_id = e.event_id
    GROUP BY
        a.athlete_id,
        e.event_name,
        e.gender
)
SELECT *
FROM ranked_athletes
ORDER BY
    event_name,
    gender,
    best_mark DESC;
"""

pd.read_sql(query_phase1, conn)

,athlete_id,athlete,event_name,gender,best_mark
0,48008,Stefka KOSTADINOVA,high jump,F,2.09
1,48009,Kajsa BERGQVIST,high jump,F,2.08
2,48010,Blanka VLAŠIĆ,high jump,F,2.08
3,48011,Lyudmila ANDONOVA,high jump,F,2.07
4,48012,Heike HENKEL,high jump,F,2.07
...,...,...,...,...,...
3678,48002,Denis SAURAMBAEV,triple jump,M,16.80
3679,48003,Aliaksandr LIABEDZKA,triple jump,M,16.80
3680,48004,Mohammed Abbas DARWISH,triple jump,M,16.80
3681,48005,José BELLIDO,triple jump,M,16.80


In [120]:
query_phase2 = """
WITH athlete_levels AS (
    SELECT
        f.athlete_id,
        e.event_name,
        e.gender,
        COUNT(DISTINCT f.age_group) AS age_levels
    FROM fact_performances f
    JOIN dim_events e
        ON f.event_id = e.event_id
    GROUP BY
        f.athlete_id,
        e.event_name,
        e.gender
    HAVING age_levels = 3
),
ranked_athletes AS (
    SELECT
        a.athlete_id,
        a.name AS athlete,
        e.event_name,
        e.gender,
        MAX(f.mark) AS best_mark
    FROM fact_performances f
    JOIN dim_athletes a
        ON f.athlete_id = a.athlete_id
    JOIN dim_events e
        ON f.event_id = e.event_id
    JOIN athlete_levels al
        ON al.athlete_id = f.athlete_id
        AND al.event_name = e.event_name
        AND al.gender = e.gender
    GROUP BY
        a.athlete_id,
        e.event_name,
        e.gender    
)
SELECT *
FROM ranked_athletes
ORDER BY 
    event_name,
    gender,
    best_mark DESC;
"""

pd.read_sql(query_phase2, conn)

,athlete_id,athlete,event_name,gender,best_mark
0,48010,Blanka VLAŠIĆ,high jump,F,2.08
1,48007,Yaroslava MAHUCHIKH,high jump,F,2.06
2,48023,Alina ASTAFEI,high jump,F,2.04
3,48034,Marina KUPTSOVA,high jump,F,2.03
4,48035,Mariya KUCHINA,high jump,F,2.03
...,...,...,...,...,...
90,47471,Jordan A. DÍAZ FORTUN,triple jump,M,18.18
91,47475,Yoelbi QUESADA,triple jump,M,17.97
92,47436,Jaydon HIBBERT,triple jump,M,17.87
93,47488,Lázaro MARTÍNEZ,triple jump,M,17.71


In [36]:
query_phase3 = """
WITH athlete_age_coverage AS (
    SELECT
        f.athlete_id,
        e.event_name,
        e.gender,
        COUNT(DISTINCT f.age_group) AS age_group_count
    FROM fact_performances f
    JOIN dim_events e
        ON f.event_id = e.event_id
    WHERE f.age_group IN ('U18', 'U20', 'SENIOR')
    GROUP BY
        f.athlete_id,
        e.event_name,
        e.gender
)
SELECT *
FROM athlete_age_coverage
WHERE age_group_count = 3;
"""

pd.read_sql(query_phase3, conn)

,athlete_id,event_name,gender,age_group_count
0,45553,high jump,M,3
1,45555,high jump,M,3
2,45557,high jump,M,3
3,45560,high jump,M,3
4,45569,high jump,M,3
...,...,...,...,...
90,48863,triple jump,F,3
91,48982,triple jump,F,3
92,48994,triple jump,F,3
93,49005,triple jump,F,3


In [51]:
query_phase4 = """
WITH eligible_athletes AS (
    SELECT
        f.athlete_id,
        f.event_id
    FROM fact_performances f
    WHERE f.age_group IN ('U18', 'U20', 'SENIOR')
    GROUP BY
        f.athlete_id,
        f.event_id
    HAVING COUNT(DISTINCT f.age_group) = 3
)

SELECT
    e.event_name,
    e.gender,
    f.age_group,
    ROUND(AVG(f.mark), 2) AS avg_mark,
    ROUND(MAX(f.mark), 2) AS best_mark,
    ROUND(MIN(f.mark), 2) AS low_mark,
    COUNT(*) AS performances
FROM fact_performances f
JOIN eligible_athletes ea
    ON f.athlete_id = ea.athlete_id
    AND f.event_id = ea.event_id
JOIN dim_events e
    ON f.event_id = e.event_id
GROUP BY
    e.event_name,
    e.gender,
    f.age_group
ORDER BY
    e.event_name,
    e.gender,
    f.age_group;
"""

pd.read_sql(query_phase4, conn)

,event_name,gender,age_group,avg_mark,best_mark,low_mark,performances
0,high jump,F,SENIOR,1.98,2.08,1.95,558
1,high jump,F,U18,1.92,1.96,1.90,60
2,high jump,F,U20,1.97,2.04,1.95,64
3,high jump,M,SENIOR,2.31,2.45,2.26,918
4,high jump,M,U18,2.27,2.33,2.25,19
5,high jump,M,U20,2.28,2.37,2.25,106
6,long jump,F,SENIOR,6.90,7.34,6.75,65
7,long jump,F,U18,6.68,7.02,6.55,15
8,long jump,F,U20,6.86,7.27,6.70,27
9,long jump,M,SENIOR,8.18,8.92,8.02,649


This query was used to ensure the number of athetes in fact_performance having performances in all three age_groups is sufficient for future analysis.

In [44]:
q = """
SELECT COUNT(*) 
FROM fact_performances f 
JOIN (
    SELECT 
        athlete_id 
    FROM fact_performances 
    WHERE age_group IS NOT NULL 
    GROUP BY athlete_id 
    HAVING COUNT(DISTINCT age_group) = 3
) ea 
ON f.athlete_id = ea.athlete_id;
"""
pd.read_sql(q, conn)

,COUNT(*)
0,3607
